In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from scipy.sparse import coo_matrix
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#os.chdir('C:\\Users\\A Sua\\Documents\\FIU\\Fall2019\\ML\\project')





"""impot data select features"""
data = pd.read_csv("application_training_data_import.csv")
features = data.columns
s=data.isna().sum()
null = pd.read_csv('Null_Entries.csv')
d = null[null['flag']==0]
columns = d['colname']
new_data = data[columns]



home_loans = new_data[new_data.isnull().any(axis=1) == False]
home_loans = home_loans.reset_index()
home_loans = home_loans.drop(columns = 'index')

#print(home_loans.shape)


cols_to_remove = ['SK_ID_CURR',
'FLAG_DOCUMENT_2','FLAG_DOCUMENT_3','FLAG_DOCUMENT_4','FLAG_DOCUMENT_5',
'FLAG_DOCUMENT_6','FLAG_DOCUMENT_7','FLAG_DOCUMENT_8',
'FLAG_DOCUMENT_9','FLAG_DOCUMENT_10','FLAG_DOCUMENT_11','FLAG_DOCUMENT_12',
'FLAG_DOCUMENT_13','FLAG_DOCUMENT_14','FLAG_DOCUMENT_15','FLAG_DOCUMENT_16',
'FLAG_DOCUMENT_17','FLAG_DOCUMENT_18','FLAG_DOCUMENT_19','FLAG_DOCUMENT_20','FLAG_DOCUMENT_21',
'AMT_REQ_CREDIT_BUREAU_HOUR',
'AMT_REQ_CREDIT_BUREAU_DAY',
'AMT_REQ_CREDIT_BUREAU_WEEK',
'FLAG_CONT_MOBILE']


cols_to_normalize = ['EXT_SOURCE_3',
'EXT_SOURCE_2','AMT_GOODS_PRICE',
'AMT_ANNUITY','DAYS_LAST_PHONE_CHANGE', 'AMT_INCOME_TOTAL',
'AMT_CREDIT','REGION_POPULATION_RELATIVE',
'DAYS_BIRTH',
'DAYS_EMPLOYED',
'DAYS_REGISTRATION',
'DAYS_ID_PUBLISH']


"""drop columns and add normalized columns"""
home_loans_df = home_loans.drop(cols_to_remove, axis=1)
## substitute the normalized columns
df_to_normalize = home_loans_df[cols_to_normalize]
norm_values = df_to_normalize.values

"""normalize values"""
scaler = StandardScaler()
scaler.fit(norm_values)
norm_values= scaler.transform(norm_values)
norm_values

"""re-impute into data"""
home_loans_df[cols_to_normalize] = norm_values
home_loans_df.head()

,EXT_SOURCE_3,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,EXT_SOURCE_2,AMT_GOODS_PRICE,...,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY
0,-1.905793,0.0,0.0,1.0,2.0,2.0,2.0,2.0,-1.351241,-0.527764,...,2,2,WEDNESDAY,10,0,0,0,0,0,0
1,1.123108,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.197991,-1.109049,...,2,2,MONDAY,9,0,0,0,0,0,0
2,0.567098,0.0,1.0,1.0,0.0,0.0,0.0,0.0,-0.868560,-0.249232,...,2,2,WEDNESDAY,16,0,0,0,0,0,0
3,-0.095792,1.0,1.0,2.0,1.0,0.0,1.0,0.0,1.086863,2.281779,...,2,2,SUNDAY,16,0,0,0,0,0,0
4,0.153597,0.0,0.0,0.0,2.0,0.0,2.0,0.0,1.035459,2.645082,...,3,3,MONDAY,16,0,0,0,0,1,1


In [2]:
"""Set the indices"""
n0 = home_loans_df[home_loans_df['TARGET']==0].shape[0]
#test set percentage size 
test_per = 0.0758
n1 = home_loans_df[home_loans_df['TARGET']==1].shape[0]
extra = 3000

train1_index = 14425
train0_index = int(train1_index)

test1_index = int(n1 - train1_index)
test0_index = int(test1_index/test_per)
#test0_index = test0_index+test1_index
# print(test1_index)
# print(test0_index)

"""Take all TARGET Home Loans"""
home_loans = shuffle(home_loans_df,random_state=0)
home_loans0 = home_loans[home_loans['TARGET']==0].iloc[:108200, :]
home_loans1 = home_loans[home_loans['TARGET']==1].iloc[:, :]

# home_loans_test0 = home_loans[home_loans['TARGET']==0].iloc[10000:101000, :]
# home_loans_test1 = home_loans[home_loans['TARGET']==1].iloc[:, :]

reduced_loans = pd.concat([home_loans0, home_loans1], axis=0)
reduced_loans = shuffle(reduced_loans,random_state=0)


reduced_loans = pd.get_dummies(reduced_loans)


##########
################
########---Split data into Test-Train Split
####################
########################
"""split dataset into training set and cross validation"""
"""training data will have 50/50 0/1 class split"""
train_df0 = reduced_loans[reduced_loans['TARGET']==0].iloc[:train0_index, :]
train_df1 = reduced_loans[reduced_loans['TARGET']==1].iloc[:train1_index, :]

train = pd.concat([train_df0, train_df1])
train = shuffle(train,random_state=43)


"""Test dataframe will have 90/10 0/1 class split"""
test_df0 = reduced_loans[reduced_loans['TARGET']==0].iloc[test0_index:, :]
test_df1 = reduced_loans[reduced_loans['TARGET']==1].iloc[test1_index:, :]

test = pd.concat([test_df0, test_df1])
test = shuffle(test,random_state=57)

del reduced_loans, home_loans, home_loans0, home_loans1
######
#########
#######-----split up the y variables
##########


"""separate out the target variable"""
train_target = train['TARGET']
train = train.drop(columns = 'TARGET')
train = pd.get_dummies(train) #One Hot Encoding
# train = train.drop(columns='NAME_INCOME_TYPE_Unemployed')
# train = train.drop(columns='GENDER_CODE_XNA')

test_target = test['TARGET']
test = test.drop(columns = 'TARGET')
test = pd.get_dummies(test) #One Hot Encoding



"""break up the data"""
X_train, y_train = np.matrix(train), np.array(train_target)
X_test, y_test = np.matrix(test), np.array(test_target)

#######
#######---Create cross validation sets-------######
########
#########

traincv1 = X_train[:int(np.floor((len(train_target)/3))), :]
target1 = y_train[:int(np.floor((len(train_target)/3)))]

traincv2 = X_train[int(np.floor((len(train_target)/3))):int(np.floor((len(train_target)*2/3))), :]
target2 = y_train[int(np.floor((len(train_target)/3))):int(np.floor((len(train_target)*2/3)))]


traincv3 = X_train[int(np.floor((len(train_target)*2/3))):, :]
target3 = y_train[int(np.floor((len(train_target)*2/3))):]


"""add test dataset"""
# test = X[int(np.floor((len(reduced_loans)*3/4))):, :]
# target_test = y[int(np.floor((len(reduced_loans)*3/4))):]


"""Trainig """
print('Training data shaped {}'.format(X_train.shape))
print(traincv1.shape)

print('Testing data shaped {}'.format(X_test.shape))


"""make pairs of datasets"""
CV12 = np.concatenate([traincv1, traincv2])
y12 = np.concatenate([target1, target2])

CV13 = np.concatenate([traincv1, traincv3])
y13 = np.concatenate([target1, target3])

CV23 = np.concatenate([traincv2, traincv3])
y23 = np.concatenate([target2, target3])

print('cross validation shapes')
print(CV12.shape)

Training data shaped (28850, 67)
(9616, 67)
Testing data shaped (61333, 67)
cross validation shapes
(19233, 67)


### Define Function

In [3]:
"""define some functions"""
def train_neural_network(nn, learning_rate, epochs, train_tensor, target_tensor): 
    """declare name, input neural_network"""
    neural_network = nn
    nn_name = nn.name
    optimizer = optim.Adam(neural_network.parameters(), lr=learning_rate) ## Adam optimizer with parameters and learning rate
    n_layers = int(str(neural_network).split('fc')[-1].split(')')[0])
    EPOCHS = epochs
    
    XX = torch.tensor(train_tensor).float()
    target = torch.tensor(target_tensor).long()
    
    for epoch in range(EPOCHS):
        XX = XX
        target = target
        
        neural_network.zero_grad()
        output = neural_network.forward(XX)
        predicted = output.float()
        
        #print(predicted.shape, target.shape)
        loss = F.cross_entropy(predicted, target)
        #loss = F.nll_loss(output, y) ## calculate loss function with MSE or cross_entropy
        #F.cross_entropy
        #  F.leaky_relu
        loss.backward() ## do the backprop
        optimizer.step() ## update the weights
        if epoch % 1000 == 0:
            print('epoch {} had loss {}'.format(epoch, loss))

    print('final loss was {}'.format(loss))
    
    nn_data = {'NN': nn_name, 'LR': learning_rate, 'Layers': 0, 'Total_Layers': n_layers, 'Epochs': EPOCHS}
#     nn_data['NN'] = nn_name
#     nn_data['LR'] = learning_rate
#     nn_data['Total_Layers']= n_layers

    layer_list = np.array([[]])
    for hidden_layer_index in range(n_layers): 
        arc = str(neural_network).split('fc')[1:][hidden_layer_index].split(',')
        in_layer = int(arc[0].split(': ')[1].split('=')[1])
        out_layer = int(arc[1].split('=')[1])
        layer_values = np.array([in_layer, out_layer])
        layer_list = np.append(arr=[layer_list], values=[layer_values])

    nn_data['Layers'] = layer_list    
    
    
    return neural_network, nn_data


In [4]:
#####
##########
###------------########
######-----------------##
#####---------------------######
def test_scores(neural_network, X_train, y_train, X_test, y_test):
    total_train = 0
    correct_train = 0
    total_test = 0
    correct_test = 0
    nn_output_train = []
    nn_output_test = []

    with torch.no_grad(): 
        XX_train = torch.tensor(X_train).float()
        XX_test = torch.tensor(X_test).float()    
        #target = torch.tensor(target_tensor).long() 

        #yy = y_train
        output_train = neural_network.forward(XX_train)
        output_test = neural_network.forward(XX_test)

        for idx, i in enumerate(output_train):
    #         print(int(torch.argmax(i)))
    #         print(type(torch.argmax(i)))
    #        print(type(output_train))
            nn_output_train.append(int(torch.argmax(i)))
            if torch.argmax(i) == y_train[idx]:
                correct_train += 1 ## check the predicted value with the actual value
            total_train += 1

        for idx, i in enumerate(output_test):
            nn_output_test.append(int(torch.argmax(i)))
            if torch.argmax(i) == y_test[idx]:
                correct_test += 1 ## check the predicted value with the actual value
            total_test += 1
            
    """Generate confusion matrices"""
    y_scores_train = np.array(nn_output_train)
    y_scores_test = np.array(nn_output_test)
    confusion_train = confusion_matrix(y_train, y_scores_train)
    confusion_test = confusion_matrix(y_test, y_scores_test)

    #print('Trained')
    train_accuracy = (confusion_train[0][0]+confusion_train[1][1])/len(y_train)
    train_recall = confusion_train[1][1]/(confusion_train[1][0]+confusion_train[1][1])
    train_precision = confusion_train[1][1]/(confusion_train[0][1]+confusion_train[1][1])                                             
    test_accuracy = (confusion_test[0][0]+confusion_test[1][1])/len(y_test)
    test_recall = confusion_test[1][1]/(confusion_test[1][0]+confusion_test[1][1])
    test_precision = confusion_test[1][1]/(confusion_test[0][1]+confusion_test[1][1])
                                                                                    
#     print('Train Accuracy: {}'.format(train_accuracy))
#     print('Train Recall: {}'.format(train_recall))
#     print('Train Precision: {}'.format(train_precision))
#     print(confusion_train)

#     print('Test Accuracy: {}'.format(test_accuracy))
#     print('Test Recall: {}'.format(test_recall))
#     print('Test Precision: {}'.format(test_precision))
    #print(confusion_test)        
    return_dic = {'Train_Size': len(y_train), 'Test_Size': len(y_test), 
                  'Train_Accuracy': train_accuracy, 'Train_Recall': train_recall, 'Train_Precision': train_precision,
                 'Test_Accuracy': test_accuracy, 'Test_Recall': test_recall, 'Test_Precision': test_precision}
    
    print('{} was tested'.format(neural_network.name))
                                           
    return confusion_train, confusion_test, return_dic

### Instantiate Neural Network

In [5]:
"""in order to make a neural network, we need to create an object belonging to the NEURAL NETWORK class"""

class Net(nn.Module):
    
    def __init__(self, name=None):
#         """inherit NN methods, attributes, functions, etc..., without this we can't use it"""
        super().__init__()
        self.fc1 = nn.Linear(in_features=1*X_train.shape[1], out_features=256) ## (each input is a 2x1 vector) 
        self.fc2 = nn.Linear(in_features=256, out_features=256) ## (layer1 output, layer 2 output)        
        self.fc3 = nn.Linear(in_features=256, out_features=64) ## (layer2 output, layer 3 output)
        self.fc4 = nn.Linear(in_features=64, out_features=2) ## (layer 3 output, layer 4 output)
        self.name = name
        self.train_size = len(y_train)
        self.test_size = len(y_test)
        print('the nnn name is {}'.format(self.name))
        
        
    def forward(self, x):
#         x = F.relu(self.fc1(x)) ## pass vector into first layer
#         x = F.relu(self.fc2(x)) ## 2nd layer
#         x = F.relu(self.fc3(x)) ### 3rd layer
        #x = F.relu
        x = F.leaky_relu(self.fc1(x), inplace=True, negative_slope=0.001)
        x = F.leaky_relu(self.fc2(x), inplace=True, negative_slope=0.001)
        x = F.leaky_relu(self.fc3(x), inplace=True, negative_slope=0.85)
        x = self.fc4(x) ### and then run softmax regression on the final layer to estimate 0,...,9 
        y = F.softmax(x, dim=1)
        #y = F.log_softmax(x, dim=1)
        return  y

### Train and perform 3-fold Cross Validation 

In [12]:
alpha = 0.005
epoch_size = 200


"""Train 12"""
nn12 = Net(name='NN12')
nn12, c12_dic = train_neural_network(nn=nn12, learning_rate=alpha, epochs=epoch_size, train_tensor=CV12, target_tensor=y12)
c12_train, c12_test, c12_scores = test_scores(neural_network=nn12, X_train=CV12, y_train=y12, X_test=traincv3, y_test=target3)

"""Train 13"""
nn13 = Net(name='NN13')    
nn13, c13_dic = train_neural_network(nn=nn12, learning_rate=alpha, epochs=epoch_size, train_tensor=CV13, target_tensor=y13)
c13_train, c13_test, c13_scores = test_scores(neural_network=nn13, X_train=CV13, y_train=y13, X_test=traincv2, y_test=target2)

"""Train 23"""
nn23 = Net(name='NN23')
nn23, c23_dic = train_neural_network(nn=nn23, learning_rate=alpha, epochs=epoch_size, train_tensor=CV23, target_tensor=y23)
c23_train, c23_test, c23_scores = test_scores(neural_network=nn23, X_train=CV23, y_train=y23, X_test=traincv1, y_test=target1)

the nnn name is NN12
epoch 0 had loss 0.6930288672447205
final loss was 0.44372084736824036
NN12 was tested
the nnn name is NN13
epoch 0 had loss 0.5475044846534729
final loss was 0.4868224263191223
NN12 was tested
the nnn name is NN23
epoch 0 had loss 0.6947623491287231
final loss was 0.4494827091693878
NN23 was tested


In [17]:
c12_scores

{'Train_Size': 19233,
 'Test_Size': 9617,
 'Train_Accuracy': 0.8730827224042012,
 'Train_Recall': 0.8721804511278195,
 'Train_Precision': 0.872818476329815,
 'Test_Accuracy': 0.6525943641468234,
 'Test_Recall': 0.6463188286244587,
 'Test_Precision': 0.6584033613445378}

In [18]:
c13_scores

{'Train_Size': 19233,
 'Test_Size': 9617,
 'Train_Accuracy': 0.8266521083554308,
 'Train_Recall': 0.8124412900532304,
 'Train_Precision': 0.8351035296641991,
 'Test_Accuracy': 0.7356764063637309,
 'Test_Recall': 0.7293559042113955,
 'Test_Precision': 0.7416036943744753}

In [19]:
c23_scores

{'Train_Size': 19234,
 'Test_Size': 9616,
 'Train_Accuracy': 0.8682541333056046,
 'Train_Recall': 0.8712472918601052,
 'Train_Precision': 0.8678450313431302,
 'Test_Accuracy': 0.6589018302828619,
 'Test_Recall': 0.6390532544378699,
 'Test_Precision': 0.6579634464751958}

### Continue training Top Performing model on Full Training Data & Test on Testing Set

In [21]:
"""Train Best"""
"""The best performing NN was the one trained on CV 1,3"""
best_nn = nn13
best_nn, best_dic = train_neural_network(nn=best_nn, learning_rate=alpha, epochs=epoch_size, train_tensor=X_train, target_tensor=y_train)
cbest_train, cbest_test, best_scores = test_scores(neural_network=best_nn, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test)

best_scores

epoch 0 had loss 0.46859070658683777
final loss was 0.4621872007846832
NN12 was tested


{'Train_Size': 28850,
 'Test_Size': 61333,
 'Train_Accuracy': 0.8512305025996534,
 'Train_Recall': 0.8465857885615251,
 'Train_Precision': 0.8545238261843118,
 'Test_Accuracy': 0.6853243767629172,
 'Test_Recall': 0.7889081455805893,
 'Test_Precision': 0.41179663470237016}